In [15]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [16]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

input_shape = x_train.shape[1:]
N_classes = 10

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, N_classes)
y_test = tf.keras.utils.to_categorical(y_test, N_classes)

170500096/170498071 [==============================] - 6s 0us/step


In [17]:
class ResnetLayer(tf.keras.Model):
  def __init__(self, 
               filters=16, 
               kernel_size=3, 
               strides=1, 
               activation='relu', 
               batch_normalization=True):
    super().__init__(name='')
    self.conv = Conv2D(filters=filters, 
                       kernel_size=kernel_size, 
                       strides=strides,
                       activation=activation,
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))
    if batch_normalization:
      self.bn = BatchNormalization()
    else: 
      self.bn = None
    if activation is not None:
      self.ac = Activation(activation)
    else:
      self.ac = None

  def call(self, input_tensor, training=False):
    x = self.conv(input_tensor)
    if self.bn is not None:
      x = self.bn(x, training=training)
    if self.ac is not None:
      x = self.ac(x)
    return x

In [18]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, 
               filters=16, 
               kernel_size=3, 
               strides=1, 
               activation='relu'):
    super().__init__(name='')
    self.rn_a = ResnetLayer(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            activation=activation)
    self.rn_b = ResnetLayer(filters=filters,
                            kernel_size=kernel_size,
                            activation=None)
    if strides != 1:
      self.skip = ResnetLayer(filters=filters,
                              kernel_size=1,
                              strides=strides,
                              activation=None,
                              batch_normalization=False)
    else:
      self.skip = lambda x: x
    if activation is not None:
      self.ac = Activation(activation)
    else:
      self.ac = None

  def call(self, input_tensor, training=False):
    x = self.rn_a(input_tensor, training=training)
    x = self.rn_b(x, training=training)
    x += self.skip(input_tensor)
    if self.ac is not None:
      x = self.ac(x)
    return x

In [19]:
def Resnet20(input_shape, N_classes): 
  return tf.keras.models.Sequential([Input(shape=input_shape), 
                                     ResnetLayer(), 
                                     ResnetIdentityBlock(), 
                                     ResnetIdentityBlock(),
                                     ResnetIdentityBlock(),
                                     ResnetIdentityBlock(filters=32, strides=2),
                                     ResnetIdentityBlock(filters=32),
                                     ResnetIdentityBlock(filters=32),
                                     ResnetIdentityBlock(filters=64, strides=2),
                                     ResnetIdentityBlock(filters=64),
                                     ResnetIdentityBlock(filters=64),
                                     AveragePooling2D(pool_size=8),
                                     Flatten(),
                                     Dense(N_classes, 
                                           activation='softmax', 
                                           kernel_initializer='he_normal')])

In [26]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

# Prepare model model saving directory.
save_dir = './drive/MyDrive/Resnet20Cifar10/History'
filepath = os.path.join (save_dir, 'epoch.{epoch:03d}.h5')

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=2,
                             save_best_only=False,
                             save_weights_only=True,)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [29]:
model = Resnet20(input_shape, N_classes)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

Learning rate:  0.001
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet_layer_63 (ResnetLayer (None, 32, 32, 16)        512       
_________________________________________________________________
resnet_identity_block_27 (Re (None, 32, 32, 16)        4768      
_________________________________________________________________
resnet_identity_block_28 (Re (None, 32, 32, 16)        4768      
_________________________________________________________________
resnet_identity_block_29 (Re (None, 32, 32, 16)        4768      
_________________________________________________________________
resnet_identity_block_30 (Re (None, 16, 16, 32)        14688     
_________________________________________________________________
resnet_identity_block_31 (Re (None, 16, 16, 32)        18752     
_________________________________________________________________
resnet_identity_block_32 (Re (No

In [28]:
History = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=200,
                    validation_data=(x_test, y_test),
                    shuffle=True,
                    callbacks=callbacks)

Epoch 1/200
Learning rate:  0.001
391/391 [==============================] - 38s 89ms/step - loss: 1.9530 - accuracy: 0.3562 - val_loss: 1.4923 - val_accuracy: 0.5124

Epoch 00001: saving model to ./drive/MyDrive/Resnet20Cifar10/History/epoch.001.h5
Epoch 2/200
Learning rate:  0.001
391/391 [==============================] - 34s 87ms/step - loss: 1.2468 - accuracy: 0.6082 - val_loss: 1.3098 - val_accuracy: 0.5887

Epoch 00002: saving model to ./drive/MyDrive/Resnet20Cifar10/History/epoch.002.h5
Epoch 3/200
Learning rate:  0.001
391/391 [==============================] - 34s 87ms/step - loss: 1.0289 - accuracy: 0.6857 - val_loss: 1.2610 - val_accuracy: 0.6175

Epoch 00003: saving model to ./drive/MyDrive/Resnet20Cifar10/History/epoch.003.h5
Epoch 4/200
Learning rate:  0.001
391/391 [==============================] - 34s 87ms/step - loss: 0.8920 - accuracy: 0.7363 - val_loss: 1.0903 - val_accuracy: 0.6778

Epoch 00004: saving model to ./drive/MyDrive/Resnet20Cifar10/History/epoch.004.h5
